# Lab4-5 (Student version): standard graph models

We can use the following libraries.

In [ ]:
import matplotlib.pyplot as plt
import math
import sys
import random
import copy
print(sys.version)

This lab work will spread over sessions 4 and 5. 

Session 4 should focus on making sure that the codes of previous sessions work correctly (ex.1) and on testing them on an Erdös-Rényi model (ex.2).

Session 5 should focus on the two other models (ex.3 and ex.4).

## Exercise 1: Preliminary work

### Question 1

Download the graph http://lioneltabourier.fr/documents/as_caida.txt and load it in memory as a dictionary of lists (as usual). This graph is a partial map of the Internet at the AS level as obtained using BGP tables during the CAIDA project in 2007. It will be used during the rest of this practical work. 

Apply the codes seen in the previous labs to:
- count its number of nodes and links, 
- plot its degree distribution,
- compute its number of triangles,
- give an approximation of its diameter.


## Exercise 2: Erdös-Rényi model

### Question 2

Create an Erdös-Rényi graph with the same number of nodes and links as the original graph.

In [ ]:
def erdos_renyi(node_count, link_count):
    graph = {}
    for node_index in range(node_count):
        graph[node_index] = []
    while link_count > 0:
        node1, node2 = random.randint(0, node_count - 1), random.randint(0, node_count - 1)
        if node1 != node2 and node1 not in graph[node2]:
            graph[node1].append(node2)
            graph[node2].append(node1)
            link_count -= 1
    return graph
        

In [ ]:
erdos_renyi(10,10)

### Question 3

Compare its degree distribution, its number of triangles, its approximate diameter (of the largest component) to the one of the original graph.

## Exercise 2: Barabasi-Albert model


### Question 4

Create a Barabasi-Albert graph with a number of links and nodes comparable to the original graph. We remind that in a BA model with $n$ nodes, the number of links $m$ is roughly equal to $\alpha n$ where $ \alpha $ is the parameter of the model. 

In [ ]:
def compute_draw_chances(graph):
    sum_degree = 0
    for node in graph:
        sum_degree += len(graph[node])
    result = {}
    print(graph)
    cumul = 1
    for node in graph:
        draw_chance = len(graph[node]) / sum_degree
        cumul -= draw_chance
        result[node] = cumul
    return result

def draw_node(draw_chances):
    tmp = random.uniform(0,1)
    for node, chance in draw_chances.items():
        if tmp > chance:
            return node
def barabasi_albert(node_count, original_graph, node_degree):
    print(original_graph)
    graph = copy.deepcopy(original_graph)
    node_count_init = len(graph)
    for node1 in range(node_count_init, node_count):
        graph[node1] = []
        link_count = 0
        draw_chances = compute_draw_chances(graph)
        while link_count < node_degree:
            tmp = random.uniform(0,1)
            node2 = draw_node(draw_chances)
            if node2 not in graph[node1]:
                graph[node1].append(node2)
                graph[node2].append(node1)
                link_count += 1
    return graph

In [ ]:
barabasi_albert(1000, erdos_renyi(10,10), 1)

### Question 5

Compare its degree distribution, its number of triangles, its approximate diameter (of the largest component) to the one of the original graph.

## Exercise 3: Watts-Strogatz model

### Question 6

Create a regular graph with a number of nodes $n$ equals to the one of the initial CAIDA graph. We have these constraints:

* all nodes of a regular graph have the same degree $k$, choose $k$ so that the number $m$ of edges is close to the one of the CAIDA graph,

* each node is connected to the nodes with the closest index, for example, if $k=6$, node $i$ will be connected to nodes $ i-1 $, $ i-2 $, $ i-3$ and $ i+1 $, $ i+2 $, $ i+3 $.  

In [ ]:
def regular(node_count, degree):
    graph = {}
    for node1 in range(node_count):
        graph[node1] = [node2 %  node_count for node2 in range(node1 - degree // 2, node1 + degree // 2 + 1)]
        graph[node1].remove(node1)
        if degree % 2 != 0:
            graph[node1].append((node1 + degree //2 + 1) % node_count)
    return graph

In [ ]:
regular(10,6)

### Question 7

Starting from the graph created in the previous question, generate Watts-Strogatz models with several values of the parameter $p$: 0.01, 0.1, 0.3.

In [ ]:
def draw_except(nodes, node):
    output = node
    while output == node:
        output = random.choice(nodes)
    return output
    

def watts_strogatz(regular_graph, rewiring_proba):
    graph = copy.deepcopy(regular_graph)
    nodes = list(regular_graph.keys())
    for node1 in regular_graph:
        for node2 in regular_graph[node1]:
            if node1 < node2:
                while True:
                    nnode1, nnode2 = node1, node2
                    if random.uniform(0,1) < rewiring_proba:
                        nnode1 = draw_except(nodes, node1)
                    if random.uniform(0,1) < rewiring_proba:
                        nnode2 = draw_except(nodes, node2)
                    if nnode1 != nnode2 and nnode1 not in graph[nnode2]:
                        graph[node1].remove(node2)
                        graph[node2].remove(node1)
                        graph[nnode1].append(nnode2)
                        graph[nnode2].append(nnode1)
                        break
                    if nnode1 == node1 and nnode2 == node2:
                        break

    return graph        

In [ ]:
watts_strogatz(regular(10,6), 0.3)

### Question 8

Compare their degree distribution, their number of triangles, their approximate diameter (of the largest component) to the one of the original graph.